In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("SQL")

LangSmith 추적을 시작합니다.
[프로젝트명]
SQL


In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# SQLite 데이터베이스에 연결합니다.
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# 데이터베이스의을 출력합니다.
print(db.dialect)

# 사용 가능한 테이블 이름들을 출력합니다.
print(db.get_usable_table_names())

sqlite
['accounts', 'customers', 'transactions']


In [3]:
# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db)

In [4]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Here is the description of the columns in the tables:
`cust`: customer name
`prod`: product name
`trans`: transaction date

Question: {input}"""
).partial(dialect=db.dialect)

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db, prompt)

In [5]:
# chain 을 실행하고 결과를 출력합니다.
generated_sql_query = chain.invoke({"question": "고객의 이름을 나열하세요"})

# 생성된 쿼리를 출력합니다.
print(generated_sql_query.__repr__())

'SELECT name\nFROM customers'


In [6]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 생성한 쿼리를 실행하기 위한 도구를 생성합니다.
execute_query = QuerySQLDataBaseTool(db=db)

C:\Users\kakao\AppData\Local\Temp\ipykernel_200\2622421503.py:4: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [7]:
execute_query.invoke({"query": generated_sql_query})

"[('테디',), ('폴',), ('셜리',), ('민수',), ('지영',), ('은정',)]"

In [8]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 도구
execute_query = QuerySQLDataBaseTool(db=db)

# SQL 쿼리 생성 체인
write_query = create_sql_query_chain(llm, db)

# 생성한 쿼리를 실행하기 위한 체인을 생성합니다.
chain = write_query | execute_query

In [9]:
# 실행 결과 확인
chain.invoke({"question": "테디의 이메일을 조회하세요"})

"[('teddy@example.com',)]"

# 답변을 LLM 으로 증강-생성

In [10]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()

# 생성한 쿼리를 실행하고 결과를 출력하기 위한 체인을 생성합니다.
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [11]:
# 실행 결과 확인
chain.invoke({"question": "테디의 transaction 의 합계를 구하세요"})

'테디의 transaction의 합계는 -965.7 입니다.'

# Agent

In [12]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# SQLite 데이터베이스에 연결합니다.
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# Agent 생성
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [13]:
# 실행 결과 확인
agent_executor.invoke(
    {"input": "테디와 셜리의 transaction 의 합계를 구하고 비교하세요"}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


accounts, customers, transactions
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	transaction_id INTEGER, 
	account_id INTEGER, 
	amount REAL, 
	transaction_date TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY(account_id) REFERENCES accounts (account_id)
)

/*
3 rows from transactions table:
transaction_id	account_id	amount	transaction_date
1	1	74.79	2024-07-13
2	1	-224.1	2024-05-13
3	1	-128.9	2024-01-25
*/
Invoking: `sql_db_query` with `{'query': 'SELECT account_id, SUM(amount) AS total_amount FROM transactions WHERE account_id IN (1, 2) GROUP BY account_id'}`


[(1, -965.7), (2, 743.13)]테디의 transaction의 합계는 -965.7이고, 셜리의 transaction의 합계는 743.13입니다.

> Finished chain.


{'input': '테디와 셜리의 transaction 의 합계를 구하고 비교하세요',
 'output': '테디의 transaction의 합계는 -965.7이고, 셜리의 transaction의 합계는 743.13입니다.'}